# Comparison of 3T and 7T data in epilepsy
-1. Concat zBrains outputs into dataframes
0. Demographics (Table 1)
1. zBrain/wBrain (surface)
    a. Histograms of vertex wise scores
        i. sub-comparisons with different smoothing kernels
    b. Quantifying extreme vertex groups
        i. number of identified abnormal areas
        ii. size of each abnormal area (number of adjacent extreme vertices)
2. Brainstats (surface)
    a. t-scores for 3T and 7T
    b. cohen's D map between 3T and 7T images

# -1. Prepare data

In [1]:
import sys
import importlib

import vrtx

sys.path.append('/host/verges/tank/data/daniel/')  # Replace with the path to Utils
from Utils import gen as gen


In [7]:
importlib.reload(vrtx)

<module 'vrtx' from '/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/Epilepsy_7T/zBrainsAnalyses/code/3T7T/analyses/vrtx.py'>

In [6]:
# IDs of interest
dir_IDs = "/host/verges/tank/data/daniel/3T7T/z/data/pt/master_demo_19Mar2025_age_y.csv" # should be in long format

# zb output directories
# N.b. .surf.gii files usually in:
# root/zBrains_output/sub-ID/ses-xx/maps/region
root_dir = "/data/mica3/BIDS_PNI/derivatives" 
dir_zb = "zbrains_3T7T_daniel_8Jan2025" # that holds sub/ses
dir_sub = "maps"

cortex = {
    "region": "cortex",
    "surfaces": ["pial", "midthickness", "white"],
    "resolution": "32k",
    "features": ["ADC", "T1map", "thickness"], # (list) features to extract
    "smoothing": [10]
    #"smoothing": [2,5,10]
}

hippocampus = {
    "region": "hippocampus",
    "surfaces": ["midthickness"],
    "resolution": "0p5mm",
    "features": ["ADC", "T1map", "thickness"], # (list) features to extract
    "smoothing": [5]
    #"smoothing": [1,2,5]
}

subcortex = {
    "region": "subcortex",
    "features": ["ADC", "T1map", "thickness"],
    "smoothing": [2,5,10]
}

regions = [cortex, hippocampus, subcortex]


In [4]:
# determine file names to look for

# file_ptrns = []  # Initialize as an empty list

# for region in regions:
#     region_ptrns = []  # Initialize as an empty list
    
#     region_ptrns = vrtx.zbFilePtrn(region)

#     # save patterns for each region in seperate list
#     file_ptrns.append(region_ptrns)  # Append directly to the list

# for region, i in zip(regions, range(len(file_ptrns))): print(f"{region['region']}: {len(file_ptrns[i])}")
# print('\n'.join(' '.join(map(str,sl)) for sl in file_ptrns))
# print() # Print the list of file patterns


cortex: 18
hippocampus: 6
subcortex: 3
hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-T1map_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-pial_feature-thickness_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-midthickness_feature-thickness_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-white_feature-ADC_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-white_feature-T1map_smooth-10mm.func.gii hemi-L_surf-fsLR-32k_label-white_feature-thickness_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-pial_feature-ADC_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-pial_feature-T1map_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-pial_feature-thickness_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-10mm.func.gii hemi-R_surf-fsLR-32k_label-midthickness_feature-T

In [8]:
# iterate through each file pattern and IDs_ses, get paths and extract paths to a df
IDs = gen.fmt(dir_IDs, ["ID", "SES"]) # read and ensure formatted ID, SES
IDs.head()

[fmt] ID formatted
[fmt] SES formatted


,ID,SES,sex,ethn,edu,hand,age_y
0,sub-HC062,ses-03,M,Mexico,PhD,R,35.08008213552361
1,sub-HC076,ses-03,F,Filipino,Grad,L,32.342231348391515
2,sub-HC081,ses-03,M,Lebanese,Undergrad,R,25.927446954141
3,sub-HC082,ses-02,F,Asian,masters,R,25.683778234086244
4,sub-HC083,ses-02,M,Mexican,Stopped during bachelors,R,28.71457905544148


In [13]:
for 

for region, region_ptrns in zip(regions, file_ptrns):
    print("================================")
    print(region["region"])
    for ptrn in region_ptrns:
        print("--------------------------------")
        ptrn_dfs = []
        for i in range(1, len(IDs)):
            ID = IDs["ID"][i]
            SES = IDs["SES"][i]
            ID_SES = ID + "_" + SES
            print(f"ID: {ID}, SES: {SES}")
            
            
            #print(ptrn)
            dir = "/".join([root_dir, dir_zb, ID, SES, dir_sub, region["region"], ID_SES + "_" + ptrn]) # read in this path, save into a column in df
            print(dir)
            # vals = vrtx.load_gifti(dir)
            # ID_ses = vrtx.get_ID_ses(dir)
            # col_name = ID_ses[0] + "_" + ID_ses[1]
            # df = vrtx.get_vrtxVals(dir)
            # ptrn_dfs.append(df)
        # save df
        

cortex
--------------------------------
hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-10mm.func.gii
ID: sub-HC076, SES: ses-03
/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC076/ses-03/maps/cortex/sub-HC076_ses-03_hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-10mm.func.gii
ID: sub-HC081, SES: ses-03
/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC081/ses-03/maps/cortex/sub-HC081_ses-03_hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-10mm.func.gii
ID: sub-HC082, SES: ses-02
/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC082/ses-02/maps/cortex/sub-HC082_ses-02_hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-10mm.func.gii
ID: sub-HC083, SES: ses-02
/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC083/ses-02/maps/cortex/sub-HC083_ses-02_hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-10mm.func.gii
ID: sub-HC088, SES: ses-02
/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC

In [ ]:
import re

# Example directory string
eg_dir = "/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC083/ses-02/maps/cortex/hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-2mm.func.gii"
print(eg_dir)

# Regular expressions to extract ID and session
match = re.search(r"sub-([^/]+)/ses-([^/]+)", eg_dir)

if match:
    ID = match.group(1)
    session = match.group(2)
    print(f"{ID}-{session}")
else:
    print("ID and session not found in the directory string.")

/data/mica3/BIDS_PNI/derivatives/zbrains_3T7T_daniel_8Jan2025/sub-HC083/ses-02/maps/cortex/hemi-L_surf-fsLR-32k_label-pial_feature-ADC_smooth-2mm.func.gii
HC083-02


## 1. zBrain
### a. Histograms
